In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!ls -lha kaggle.json

In [3]:
#!pip install -q kaggle

In [4]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/

In [5]:
#!chmod 600 /root/.kaggle/kaggle.json

In [6]:
#!pwd

/content


In [13]:
#!kaggle datasets list

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          13294        273  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51          10089        211  1.0              
ambaliyagati/spotify-dataset-for-playing-around-with-sql            Spotify dataset                                     302KB  2024-06-17 10:48:20           1978         43  1.0              
ihelon/coffee-sales                     

In [ ]:
#!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

In [ ]:
#!unzip amazon-books-reviews.zip

## **File Reading Approaches**

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **Pandas:**

In [132]:
df_ratings = pd.read_csv('Books_rating.csv')
df_books = pd.read_csv('books_data.csv')

> Pandas took almost ~1 minute to load the data.

In [131]:
# Deleting the dataframes

lst = [df_ratings, df_books]
del lst
del df_ratings, df_books

### **Files:**

In [129]:
with open('Books_rating.csv', 'r') as f:
  df_ratings = f.read()

with open('books_data.csv', 'r') as f:
  df_books = f.read()

> Reading by open() & read() function takes ~ 15 seconds to read the files. Which is an improvement over the Pandas library.

In [124]:
with open('Books_rating.csv', 'r') as f:
  df_ratings = f.readlines()

with open('books_data.csv', 'r') as f:
  df_books = f.readlines()

> - Reading by open() & readlines() function takes ~ 5 seconds to read the files. Which is much faster compared to read() function and Pandas library.
> - However, readlines() method returns a list where each item of the list is a complete sentence in a file. Since it appends each line to the list and then returns the entire list it will be time consuming if the file size is quite large.

### **Creating Utility File for data ingestion:**

In [121]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import datetime
import gc
import re

def read_config_file(filepath):
  with open(filepath, 'r') as stream:
    try:
      return yaml.safe_load(stream)
    except yaml.YAMLError as exc:
      logging.error(exc)

def replacer(string, char):
  pattern = re.escape(char) + '{2,}'
  string = re.sub(pattern, char, string)
  return string

def col_header_val(df, table_config):
  '''
  Replace whitespaces in column and
  Standardize column names.
  '''
  df.columns = df.columns.str.lower()
  df.columns = df.columns.str.replace('[^\w]', '_', regex = True)
  df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
  df.columns = list(map(lambda x: x.replacer(x, '_'), list(df.columns)))
  expected_col = list(map(lambda x: x.lower(), table_config['columns']))
  expected_col.sort()
  df.columns = list(map(lambda x: x.lower(), list(df.columns)))
  df = df.reindex(sorted(df.columns), axis = 1)
  if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
    print("column name and column length validation passed")
    return 1
  else:
    print("column name and column length validation failed")
    mismatched_columns_file = list(set(df.columns).difference(expected_col))
    print("Columns not in YAML files", mismatched_columns_file)
    missing_YAML_file = list(set(expected_col).difference(df.columns))
    print("Columns not in uploaded file", mismatched_YAML_file)
    logging.info(f'df columns: {df.columns}')
    logging.info(f'expected columns: {expected_col}')
    return 0


Overwriting utility.py


In [135]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - Id
    - Title
    - Price
    - User_id
    - profileName
    - review/helpfulness
    - review/score
    - review/time
    - review/summary
    - review/text

Writing file.yaml


### **Parameterizing process using YAML file:**

In [122]:
import utility as util